In [1]:
#!pip install selenium
#!pip install geonamescache
from helper import *
import selenium
import tweepy
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from IPython.display import clear_output #clears output
import datetime
import time

API_Key ='irXxqVrXin1KsDRM9g6HdF8p8'
API_Key_Secret = 'sLQvb8voyrTDz5S93vlt5jE5RlMi8xBuOQtSq2pgERe2QVawIC'

Bearer_Token = 'AAAAAAAAAAAAAAAAAAAAACsVlAEAAAAADib49u9Asa6Hbts8UwBEdtc0dsk%3D1d8lVaySeRg1CpJQ6HDLSwK6Mj9wJiptj49PAdwMuvW95zJoar'

Access_Token = '1122492746210660352-COptyz3RtF1AVokHw3027iOdbsZZl1'
Access_Token_Secret = 'slfBz8pcgCrUmewZgSgVKsmIFhfCcFuinAx16q5eh1Vvr'

In [2]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
# set access to user's access key and access secret
auth.set_access_token(Access_Token, Access_Token_Secret)
# calling the api
api = tweepy.API(auth)

Client = tweepy.Client(bearer_token=Bearer_Token) 


In [3]:
'''
#tweets_list = api.search_30_day(label = '30day',query = '#blender', fromDate = FROM_DATE,toDate = TO_DATE,maxResults = 10,tweet_mode = "extended")
#tweets_list = tweepy.Cursor(api.search_tweets, "biden", count=100).items(100)


start_date = datetime.datetime.now() - datetime.timedelta(days=14)
end_date = start_date + datetime.timedelta(days=7)

start_date = datetime.datetime.now()
end_date = start_date + datetime.timedelta(days=1)
'''
trends = getAllTrends()#get trends

# Search for tweets within the target countries
usa = '96683cc9126741d1'
canada = "3376992a082d67c7"
great_britain = '6416b8512febefc9'
Australia = '3f14ce28dc7c4566'
new_zealand = "0b039ce18ddf1cb9"

tweets_list = []
#ALL_pages = tweepy.Cursor(api.search_tweets, q="since:{} until:{}".format(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")), lang="en", tweet_mode="extended", count = 100).pages(50)
max_count_4_each_country = 10

#add usa
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ usa + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('Us secured')
time.sleep(10)

#add Canada
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ canada + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('canada secured')
time.sleep(10)

#add "Great" Britian
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ great_britain + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('UK secured')
time.sleep(10)

#add Australia
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ Australia + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('kengooro land secured')
time.sleep(10)

#add new zealand
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ new_zealand + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print("kengooroon't land secured")


'''
#return this if you are using pages again!!!!!
for page in ALL_pages:
    tweets_list.extend(page)
#tweets_list = [t for t in tweets_list if t.in_reply_to_status_id == None]    
'''
print('done')

Us secured
canada secured
UK secured
kengooro land secured
kengooroon't land secured
done


In [4]:
tweets_list = [t for t in tweets_list]
len(tweets_list)

50

In [10]:
#search_tweets(q, *, geocode, lang, locale, result_type, count, until, since_id, max_id, include_entities)

#user data

DRIVER = webdriver.Chrome()#the chrome

user_name = [];
#location = [];

city = [];
country = [];
age = [];
gender = [];
followers = [];
following = [];
verified = []
joined = [] #when they joined
respectability = []
tweet_count_user = []

#tweet data
tweet = [];
word_count = [];
likes = []
retweet_count = []
is_quote_status = []
date = []
tweet_id = []
hashtags = []
hashtags_count = []
trending_for_area = []

users = [];

print('loading users')

arr = []
i = 1

for t in tweets_list:
    tweet.append(t.full_text);
    
    users.append(t.user);
    
    word_count.append(countWordsInString(t.full_text))
    
    likes.append( t.favorite_count)
    
    retweet_count.append(t.retweet_count)
    
    is_quote_status.append(t.is_quote_status)
    
    date.append(t.created_at.isoformat()[:10])
    
    tweet_id.append(t.id)
    
    ht = [t['text'] for t in t.entities['hashtags']]
    ht_count = len(ht)
    
    if ht == []:
        ht = np.nan
        ht_count = 0
    
    hashtags.append(ht)
    hashtags_count.append(ht_count)
    
    
    city.append(t.place.name)
    country.append( t.place.country)
    
    trending_for_area.append(getTrendsByLoc(t.place.name,t.place.country,trends))
        
for x in users:
    if i%26 == 0:
        print("waiting")
        time.sleep(6)
    j = x._json
    
    #opens the page
    screen_name = j['screen_name']    
    DRIVER.get('https://twitter.com/' + screen_name) 
    
    #get the easy stuff
    user_name.append(screen_name)
    following.append(j['friends_count'])
    followers.append(j['followers_count'])
    
    respectability.append(float(j['friends_count']) / float(j['followers_count']))
    joined.append(x.created_at.isoformat()[:10])
    tweet_count_user.append(x.statuses_count)
    verified.append(x.verified)
    
    
    
    tmp = getAge(DRIVER)#gets the age
    if(tmp == 2023 - x.created_at.year): #checks if the age is the same as the accounts age
        age.append(np.nan)
    else:
        age.append(tmp)
    
    
    gender.append(getGender(j['description'],j['location'],j['name']))
    
    ##this part simply displays at the bottom how many profiles were checked
    clear_output(wait=True) #clears the output
    
    i+=1
    print(i)

print('done')

51
done


In [11]:
max = len(users)

In [12]:
dick = {"name":user_name[:max],
        "age":age[:max],
        'city':city[:max],
        'country':country[:max],
        "gender":gender[:max],
        'join_date':joined[:max],
        'total tweets': tweet_count_user[:max],
        "followers":followers[:max],
        "following":following[:max],
        'respectability':respectability[:max],
        'verified' : verified[:max],
        'tweet':tweet[:max],
        'likes':likes[:max],
        'retweets':retweet_count[:max],
        'word count': word_count[:max],
        'is quote':is_quote_status[:max],
        'hashtags':hashtags[:max],
        'hashtag count':hashtags_count[:max],
        'trending': trending_for_area[:max],
        'date':date[:max],
        'tweet id':tweet_id[:max]
       }

df = pd.DataFrame(dick)

In [13]:
df

,name,age,city,country,gender,join_date,total tweets,followers,following,respectability,...,tweet,likes,retweets,word count,is quote,hashtags,hashtag count,trending,date,tweet id
0,TrustinHerbert,NaN,Fair Oaks,United States,None,2020-08-27,15921,1935,2357,1.218088,...,Ragetweet... as opposed to retweet https://t.c...,0,0,6,True,NaN,0,"[Friday the 13th, Lori Harvey, Brandon Staley,...",2023-01-13,1613957669181886464
1,thechub_b_w,NaN,Houston,United States,None,2021-02-11,113000,1632,1877,1.150123,...,I’m her twin https://t.co/Iqcr3HuLnP https://t...,0,0,5,True,NaN,0,"[GAME DAY, #GoSpursGo, Lisa Marie, Elvis, Frid...",2023-01-13,1613957666946322432
2,peacelovediam,NaN,Roswell,United States,None,2011-06-29,22016,929,665,0.715823,...,Some of y’all really do hate reciprocity. &amp...,0,0,10,False,NaN,0,"[Friday the 13th, Lori Harvey, Brandon Staley,...",2023-01-13,1613957666132328455
3,tmj_MD_pharm,NaN,Oakland,United States,None,2010-02-11,144,260,241,0.926923,...,"Interested in a job in Oakland, MD? This could...",0,0,28,False,"[CVSPharmTech, PharmTech]",2,"[Friday the 13th, Lori Harvey, Brandon Staley,...",2023-01-13,1613957664375177216
4,_mariacollins,NaN,Lake Forest,United States,None,2021-05-24,887,457,94,0.205689,...,2023 surround yourself with good human beings,0,0,7,False,NaN,0,"[Friday the 13th, Lori Harvey, Brandon Staley,...",2023-01-13,1613957663867678726
5,SHS_BasketbaII,NaN,Indiana,United States,None,2014-03-18,1098,599,77,0.128548,...,Home game tonight vs Seeger. Youth league intr...,0,0,13,False,[family],1,"[Friday the 13th, Lori Harvey, Brandon Staley,...",2023-01-13,1613957660503588866
6,ARMY19562,NaN,Dallas,United States,None,2022-08-22,21876,2064,5001,2.422965,...,YELLEN SAYS US TO TAKE \nEXTRAORDINARY STEPS T...,0,0,12,False,[BLOOMBERG],1,"[Friday the 13th, Lori Harvey, Brandon Staley,...",2023-01-13,1613957659304071168
7,MoondogGrowlers,NaN,Dunwoody,United States,None,2012-01-10,5784,897,299,0.333333,...,Just posted a photo @ Moondog Growlers https:/...,0,0,8,False,NaN,0,"[Friday the 13th, Lori Harvey, Brandon Staley,...",2023-01-13,1613957657349718016
8,erikarenee21,NaN,San Antonio,United States,None,2019-12-02,238,200,634,3.170000,...,Muchísimas gracias 🙏🏼 a @sologruperas por publ...,0,0,29,False,"[sologruperas, erikarenée, acércate, nuevoestr...",7,"[GAME DAY, #GoSpursGo, Trumps, #OpenDnD, #PorV...",2023-01-13,1613957657320394753
9,KatrinaMarie_3,NaN,Buffalo,United States,None,2022-07-28,561,21,23,1.095238,...,I can love me better! ✌🏻\nI do not own copyrig...,0,0,20,False,"[Flowers, Music, MileyCyrus, ICanLoveMeBetter]",4,"[Friday the 13th, Lori Harvey, Brandon Staley,...",2023-01-13,1613957656481497089


In [14]:
DRIVER.quit()

In [15]:
df.to_csv('tmp.csv')